# Taking initiative

Having solved her basic needs for food, dr. Elena Vasilescu started worrying about other primitive survival needs of her body: heat! The base was relying on solar panels for generating heat. However, she would be freezing to death whenever the first sandstorm would hit the region. Sandstorms are pretty often on that corner of the solar system. The initial plan was for the team to set up additional support systems that would store excessive energy in batteries, but those modules have been lost during the landing failure.

She had to get her hands on a reliable heat source to keep her self warm during the cold days. Luckily, the engineers back at home had prepared the mission with such a source, albeit to be used only in extreme emergency situations.

<img src="../artwork/stranded/elena_teleoperating_robot.png" width=60%/>

Not being able to contact the Earth, Dr. Vasilescu took the initiative to consider herself in an extreme emergency situation.

## Robot modelling

In the [previous laboratory](lab01_Transformations.ipynb), we explored the concepts of coordinate frames and transformations, in the form of matrices. Even though the concepts might seem too fundamental, they have direct applications in robotics and a proper understanding of them is essential when modelling the kinematics of robot systems. This is true for any kind of robot, whether it is stationary or moving, flying, walking, rolling, or swimming.

In the current laboratory, we will apply this knowledge in the description of the kinematics of articulated robots (also referred to as robot arms). But let's first understand what we mean by a robot arm.

### Anatomy of a Robot

An articulated robot, is a mechanism consisting of two main elements: links and joints. The __links__ of the robot are the elements that provide the robot with its structure. Typically, they are very stiff (except for flexible robots), and for our modelling work they are considered _non-deformable_. The __joints__ of the robot are the elements that provide it with its flexibility. They connect two links together, and allow them to move relative to each other.

There are two types of joints that are typically used in robots: __Revolute__ and __Prismatic__. A revolute joint is a joint that allows rotation between two links about a certain axis of rotation. 

<img src="../artwork/fkine/revolute_joint.png" width=60%/>

A prismatic joint is a joint that allows translation between two links along a certain axis of translation.

<img src="../artwork/fkine/prismatic_joint.png" width=60%/>

Each of these types of joints add flexibility to the robot, allowing its end-effector to reach various positions and orientations. The way links and joints are combined affect the _degrees of freedom_ and the work envelope of the robot.

The _work envelope_ is the volume around the robot that the end-effector can reach. The shape of the work envelope depends on the size of the links and the configuration of the joints, ranging from a rectangular parallelepiped for Cartesian robots, to more complex volumes for articulated robots.

<img src="../artwork/fkine/work-envelope.jpg" width=60%/>

The _degrees of freedom_ refer to the number of coordinates of the end-effector that the robot is able to change __independently__. In 3-dimensional space, we can have up to six degrees of freedom (six different coordinates that we can change independently): three translations and three rotations. However, not every robot allows all six degrees of freedom to their end-effector; it depends on the number and configuration of the joints. Most modern articulated robots, though, do provide six degrees of freedom at the end-effector.

The degrees of freedom of the robot depend also on the area of the work envelope. Usually, when the end-effector is on the limits of the work envelope it looses flexibility and is therefore loosing some degrees of freedom. We will discuss more about this on the [Jacobian laboratory](lab04_Jacobian.ipynb).

## Forward kinematics

One basic challenge when working with robot systems is to determine the pose (position and orientation) of a certain part of interest of the robot, w.r.t. a reference frame. Most often, the point of interest of the robot is the end-effector, or the tip of the robot. There, we typically attach a tool or a sensor that we need to know its pose.

To be able to describe pose, we use the tools that we already learned in the [Transformations laboratory](lab01_Transformations.ipynb). More specifically, we use coordinate frames and transformation matrices, since transformation matrices can describe the pose of a target frame w.r.t. a reference frame.

To do so, we perform the following steps:

* We decide on a reference frame $b$, w.r.t. which we will calculate the pose of the end-effector
* We identify the links and joints of the robot
* We attach a coordinate frame $i$ on the $i^{th}$ link
* We attach a coordinate frame $n$ on the end effector

Our task now is to calculate the transformation matrix $T_b^n$ that describes the pose of the end-effector frame $n$, w.r.t. the reference frame $b$. This is possible, if we can calculate the intermediate transformations $T_b^1$, $T_1^2$, ..., $T_{n-1}^n$, and then combine them to calculate the overall transformation.

### 3-joint planar robot example, static configuration

In the figure below, you can see a robot with three revolute joints, all three rotating around axes perpendicular to the screen surface. This is called a 3-joint planar robot, as all parts of the robot will be constantly on the same plane.

<img src="../artwork/fkine/manipulator_with_cof.png" width=60%/>

On this robot, we have attached a reference frame $0$ (black frame), and an end-effector frame $4$ (purple frame). We have also attached reference frames at all intermediary links. The following transformations are described between subsequent frames:

* $T_0^1 = Tz(1)$
* $T_1^2 = Tz(2)$
* $T_2^3 = Ty(3)$
* $T_3^4 = Tz(-1)$

Each one of this transformation is describing the pose of the frame mentioned in the superscript w.r.t. the frame mentioned in the subscript. To calculate the overall transformation, we multiply the subsequent transformations with _relative_ multiplication.

$$ T_0^4 = T_0^1 T_1^2 T_2^3 T_3^4 = \begin{bmatrix}
   1 & 0 & 0 & 0  \\
   0 & 1 & 0 & 3  \\
   0 & 0 & 1 & 2  \\
   0 & 0 & 0 & 1  \\
\end{bmatrix}$$

### 3-joint planar robot example, dynamic configuration

Of course, robots are not stationary and don't always have the same configuration, but they are moving due to joint motion. To calculate the forward kinematics for any robot joint configuration, we follow the same approach. In the following figure, we see the same robot, but after the joints have moved by $q_1$, $q_2$, and $q_3$ degrees.

<img src="../artwork/fkine/manipulator_with_cof2.png" width=60%/>

The frames attached to the links have rotated with them, and the transformations between them have changed. Our task is now to describe these transformations in terms of $q_1$, $q_2$, and $q_3$. These three variables are what we call the __joint coordinates__, or __generalized coordinates__ of the robot.

To calculate the forward kinematics of this robot, which is the transformation of frame $4$ w.r.t. frame $0$, we proceed as earlier: We calculate the transformations between subsequent frames, and then we multiply them to calculate the overall transformation.

The transformation from frame $0$ to frame $1$ is the result of two elementary transformations, one rotation and one translation. To calculate it, we can imagine what kind of transformations do we need to _apply_ to frame $0$ so that it reaches the pose of frame $1$. There are two ways to calculate this:

* We can first consider that frame $0$ translates on the $Z$ axis by 1 unit, and then rotates around the translated $X$ axis by $q_1$ degrees. In this case, the second transformation needs to be multiplied from the right of the first, since it is a relative transformation:

$$ T_0^1 = Tz(1)Rx(q_1) $$

* We can first consider that frame $0$ rotates around the $X$ axis by $q_1$, and then translates around the original $Z$ axis by one unit. In this case, the second transformation needs to be multiplied from the left of the first, since it is an absolute transformation:

$$ T_0^1 = Tz(1)Rx(q_1) $$

As you can observe, both of these approaches result in the same transformation matrix. PLay with the values of the following widget to get more familiar with absolute and relative transformations.

In [15]:
from ipywidgets import interact, widgets
from spatialmath.base import *
import numpy as np
    
def complexTransform1(a,b):
    t = transl(0,0,a)@trotx(np.radians(b))
    trplot(trotx(0), color='gray', width=1, frame='B', length=1.3)
    trplot(t, frame='A', style='rviz', width=2, dims=[-2, 2, -2, 2, -2, 2])
    print('Tz*Rx = \n',t)

def complexTransform2(a,b):
    t = trotx(np.radians(b))@transl(0,0,a)
    trplot(trotx(0), color='gray', width=1, frame='B', length=1.3)
    trplot(t, frame='A', style='rviz', width=2, dims=[-2, 2, -2, 2, -2, 2])
    print('Rx*Tz = \n',t)

ctr = widgets.interactive(complexTransform1, a=(-3.0,3.0), b=(-90,90))
ctl = widgets.interactive(complexTransform2, a=(-3.0,3.0), b=(-90,90))

tab  = widgets.Tab(children = [ctr, ctl])
tab.set_title(0, 'Tz*Rx')
tab.set_title(1, 'Rx*Tz')

display(tab)

Following the same logic as for the first link, we can calculate the transformation between the next frames as following:

* $T_1^2 = Ty(l_1)Rx(q_2) = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & c_2 & -s_2 & l_1 \\ 0 & s_2 & c_3 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}$
* $T_2^3 = Ty(l_1)Rx(q_3) = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & c_3 & -s_3 & l_2 \\ 0 & s_3 & c_3 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}$
* $T_3^4 = Ty(l_3) = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & l_3 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}$

For printing efficiency, we will use the following notation:

\begin{equation}
\left\{ {{\begin{array}{rcl}
 c_i\stackrel{}{=}  & cos(q_i)  &     \\
 s_i\stackrel{}{=}  & sin(q_i) &    \\
 c_{i,j}\stackrel{}{=} & cos(q_i+q_j) & \\
 s_{i,j}\stackrel{}{=} & sin(q_i+q_j) & \\
 \end{array} }} \right., for\;any\; generalised\;coordinate\; q_i,\; i=\overline{1..n}
\end{equation}

Therefore, the forward kinematics of the robot is calculated by multiplying the subsequent transformation matrices, so that you calculate the overall transformation from frame $0$ to frame $4$:

$$ T_0^4 = T_0^1T_1^2T_2^3T_3^4 = \begin{bmatrix}
    1 & 0 & 0 & 0 \\
    0 & c_{1,2,3} & -s_{1,2,3} & l_3c_{1,2,3} + l_2c_{1,2} + l_1c_1 \\
    0 & s_{1,2,3} & c_{1,2,3}  & l_3s_{1,2,3} + l_2s_{1,2} + l_1s_1 + 1\\
    0 & 0 & 0 & 1
\end{bmatrix} $$

## Robotic toolbox

### Symbolic transformation matrices

Using the robotics toolbox, you can calculate transformation matrices with symbolic expressions, e.g., for the joint coordinates. You can also combine several symbolic transformations to calculate an overall symbolic transformation (e.g., for the forward kinematics).

In [ ]:
from spatialmath.base import *
from spatialmath import *
import spatialmath.base.symbolic as sym
from sympy import *

# Defining the joint coordinates symbols
q0, q1, q2, q3, q4, q5, l1, l2 = sym.symbol('q0, q1, q2, q3, q4, q5, l1, l2')

# Calculating the symbolic forward kinematics model
dgm = trotx(-pi/2)@trotz(q1)@trotx(pi/2)@trotz(q2)@trotx(-pi/2)@trotz(q3)@trotx(pi/2)@trotz(q4)@transl(0,0,l1)@trotx(-pi/2)@trotz(q5)@transl(l2,0,0)
print(dgm)

# Calculating the forward kinematics model for specific joint coordinates
#Matrix(dgm).subs({q1:pi, q2:pi/2, q3:pi/2, q4:pi/5, q5:0})

### Robot models

The toolbox is preloaded with definitions of the models of several popular robots. You can use these models to calculate several aspects of the robot, such as e.g. the forward kinematics for joint coordinates. You can do that following the example below

In [ ]:
import roboticstoolbox as rtb
import numpy as np

robot = rtb.models.UR5() # This is the gray-blueish robot in the C12 laboratory
print(robot) # Print details of the robot

q = np.radians([0, 0, 0, 0, 0, 0]) # The 6 joint coordinates (degrees, converted to radians) corresponding to the upright configuration of the robot

fk = robot.fkine(q)
print(fk)

## Helping dr. Vasilescu

Now that you know enough about forward kinematics, go ahead and help dr. Vasilescu [access a reliable heat source](../assignments/Assignment02_ForwardKinematics.ipynb)